In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# Initializing our train and validation folders' path to a variable
train_dir = "covid19_prediction/train/"
val_dir = "covid19_prediction/val/"

In [3]:
import pathlib

In [4]:
# Making our path to a python accessable path
data_dir = pathlib.Path("covid19_prediction/train/") 
data_dir

WindowsPath('covid19_prediction/train')

In [5]:
# Getting our class names for prediction
class_names = np.array([item.name for item in data_dir.glob('*')])
class_names

array(['Not_Covid', 'Viral'], dtype='<U9')

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed
tf.random.set_seed(42)

# Preprocess data 
train_datagen = ImageDataGenerator(rescale=1./255, # Get the pixel values between 0 and 1 --> Scaling
                                  rotation_range=20, # The next few hyper-params are used for used for data augmentation so that our model generalizes on the testing data
                                  width_shift_range=0.2,
                                  horizontal_flip=True,
                                  zoom_range=0.2,
                                  shear_range=0.2,
                                  height_shift_range=0.2)
valid_datagen = ImageDataGenerator(rescale=1./255)


# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary",# Type of data to feed into our model
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(val_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)



Found 2002 images belonging to 2 classes.
Found 988 images belonging to 2 classes.


In [7]:
def plot_curves(model):
    loss=model.history['loss']
    val_loss = model.history['val_loss']
    accuracy = model.history['accuracy']
    val_accuracy = model.history['val_accuracy']
    epochs=range(len(model.history['loss']))
    
    plt.plot(epochs,loss,label='training_loss')
    plt.plot(epochs,val_loss,label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    
    plt.figure()
    plt.plot(epochs,accuracy,label='Training_Accuracy')
    plt.plot(epochs,val_accuracy,label='Validation_Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();
    

In [9]:
import tensorflow_hub as hub

In [10]:
!pip install tensorflow_hub

In [11]:
url='https://tfhub.dev/tensorflow/efficientnet/b7/classification/1'

feature_extractor = hub.KerasLayer(url,
                                  trainable=False,
                                  input_shape=(224,224,3))

model = tf.keras.Sequential([feature_extractor,
                            tf.keras.layers.Dense(1,activation='sigmoid')
])

OSError: SavedModel file does not exist at: C:\Users\AKSHAT~1\AppData\Local\Temp\tfhub_modules\c2f1f54e53e4a0cb6f0cda17772d576872d75185\{saved_model.pbtxt|saved_model.pb}

In [12]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
fit_model=model.fit(train_data,
                    epochs=5,
                   steps_per_epoch=len(train_data),
                   validation_data=valid_data,
                    validation_steps=len(valid_data))